# Deployment management in Prefect

There are many ways to create deployment in Prefect
- Web UI
- python SDK
- YAML config file

## 1. Create a deployment with python sdk

With python sdk, we can use two methods to create a deployment:
  - `flow.serve()` : simple to implement, but not support dynamic dispatch. (Not recommended)
  - `flow.deploy()`: enables dynamic dispatch. (Recommended)

### 1.1 Create a deployment with flow.deploy()

Before you start, ensure you have the following:

1. `A running Prefect server`
2. `A Prefect flow`
3. `A work pool`
4. `A running worker` attached to the work pool

Let's use the below workflow as base workflow

```python
from prefect import flow, task
from prefect.artifacts import create_markdown_artifact


# common function
def get_words(input_str: str) -> list[str]:
    # Strip leading/trailing spaces, then split by space
    words = input_str.strip().split(" ")
    # Filter out empty entries (for consecutive spaces)
    return [w for w in words if w]


# ----- LEVEL 1: Base Tasks -----
@task(name="Task_1", description="task 1 will count total words", log_prints=True)
def task1(input_str: str) -> int:
    print("Running Task 1")
    words = get_words(input_str)
    print("Completed Task 1")
    return len(words)


@task(name="Subtask_2a", description="task 2a will convert string to a list of words", log_prints=True)
def subtask2a(input_str: str) -> list[str]:
    print("Running Subtask 2a")
    return get_words(input_str)


@task(name="Subtask_2b", description="task 2b will remove duplicates in the list", log_prints=True)
def subtask2b(words: list[str]) -> list[str]:
    print("Running Subtask 2b")
    seen = set()
    unique_words = []
    for word in words:
        if word not in seen:
            unique_words.append(word)
            seen.add(word)
    return unique_words


@task(name="Task_3", description="show a report with total words and words list", log_prints=True)
def task3(result1, result2):
    print(f"Running Task 3 after Task 1 and Task 2")
    markdown_report = f"""# Book processing report

## Summary

In this report, we count total words and show unique words list.

| word count        | unique word list |
|:--------------|-------:|
| {result1} | {result2} |
"""
    create_markdown_artifact(
        key="word-report",
        markdown=markdown_report,
        description="book processing report",
    )
    print("Completed Task 3")


# ----- LEVEL 2: Task 2 as a Subflow -----
@flow(name="subflow_for_task_2", log_prints=True)
def task2(input_str: str) -> list[str]:
    print("Starting Task 2 Flow")
    res2a = subtask2a(input_str)
    res2b = subtask2b(res2a)
    print("Completed Task 2 Flow")
    return res2b


# ----- LEVEL 3: Main Orchestration Flow -----
@flow(name="Flow_with_artifact", description="This workflow read a book, then show total words count and unique words",
      version="1.0.0", log_prints=True)
def main_flow(book_str: str) -> None:
    print("=== Main Flow Started ===")

    # Run the first task
    result1 = task1(book_str)
    # Run the second `task`
    # you can notice task2 is actual a flow
    # When you call a flow inside another flow, it becomes subflow
    result2 = task2(book_str)

    # Task 3 depends on both Task 1 and Task 2
    task3(wait_for=[result1, result2], result1=result1, result2=result2)

    print("=== Main Flow Completed ===")


if __name__ == "__main__":
    test_str = "people needs to eat more fruits. people needs to do more sports"
    main_flow(test_str)

```

We need to rewrite the main method.

```python
if __name__ == "__main__":
    test_str = "people needs to eat more fruits. people needs to do more sports"
    (flow.from_source(
        source="C:/Users/PLIU/Documents/git/Seminar_workflow_automation/projects/04_deployment_in_prefect/flows",
        entrypoint="deploy_flow_with_params.py:main_flow")
     .deploy(name="deploy_flow_with_params", description="This workflow test task and flow with various parameters",
             work_pool_name="pliu-pool", parameters={"book_str": test_str}))
```

> there are two sections in the deployment description
> - from_source(): specify where is the code source
> - deploy(): specify the deployment parameters

```powershell
# to register the deployment, you need only run
python .\deploy_flow_with_params.py

# expected output
# Looks like you're deploying to a process work pool. If you're creating a deployment for local development, calling `.serve` on your flow is a great way to get started. See the documentation for more information: https://docs.prefect.io/latest/how-to-guides/deployments/run-flows-in-local-processes  Set
# `ignore_warnings=True` to suppress this message.
# Successfully created/updated all deployments!

                          Deployments
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Name                                     ┃ Status  ┃ Details ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ Flow_with_params/deploy_flow_with_params │ applied │         │
└──────────────────────────────────────────┴─────────┴─────────┘

# To schedule a run for this deployment, use the following command:

prefect deployment run 'Flow_with_params/deploy_flow_with_params'
```

### 1.2 Add a scheduler to the deployment

You can notice, to run the above deployment, we need to trigger it manually. To allow Prefect server to trigger a deployment run, we need to add a scheduler.

Prefect provides the below schedulers:
- **interval**: Defines the interval at which the flow should run.
- **cron**: Defines a cron string at which the flow should run.
- **rrule**: Defines a rrule string. For example, "FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;BYHOUR=9;BYMINUTE=0" Run every weekday at 9 AM
- `schedules`: Advance scheduler setup which contains various recurring schedules, complex scheduling logic, custom  timezone offsets

from platform import python_branch

#### 1.2.1 Add Interval scheduler to the deployment

Let's reuse the previous example, in the below code, we just added an `interval scheduler`

```python
if __name__ == "__main__":
    test_str = "people needs to eat more fruits. people needs to do more sports"
    (flow.from_source(
        source="C:/Users/PLIU/Documents/git/Seminar_workflow_automation/projects/04_deployment_in_prefect/flows",
        entrypoint="deploy_flow_with_interval_scheduler.py:main_flow")
             # name of the deployment
     .deploy(name="deploy_flow_with_interval_scheduler",
             # description of the deployment
             description="This workflow test task and flow with various parameters",
             # which work pool the deployment will be deployed
             work_pool_name="pliu-pool",
             # workflow parameters
             parameters={"book_str": test_str},
             # interval scheduler, it will trigger the workflow evey 3 mins
             interval=timedelta(minutes=3)
             ))
```

#### 1.2.2 Add a cron scheduler to the deployment

Let's reuse the previous example, in the below code, we just added an `cron scheduler`

```python
if __name__ == "__main__":
    test_str = "people needs to eat more fruits. people needs to do more sports"
    (flow.from_source(
        source="C:/Users/PLIU/Documents/git/Seminar_workflow_automation/projects/04_deployment_in_prefect/flows",
        entrypoint="deploy_flow_with_cron_scheduler.py:main_flow")
     .deploy(name="deploy_flow_with_cron_scheduler",
             description="This deployment test cron scheduler",
             work_pool_name="pliu-pool",
             parameters={"book_str": test_str},
             # Run once a day at midnight
             cron="0 0 * * *"
             ))
````

#### 1.2.2 Add a rrule scheduler to the deployment

Let's reuse the previous example, in the below code, we just added an `rrule scheduler`

```python
if __name__ == "__main__":
    test_str = "people needs to eat more fruits. people needs to do more sports"
    (flow.from_source(
        source="C:/Users/PLIU/Documents/git/Seminar_workflow_automation/projects/04_deployment_in_prefect/flows",
        entrypoint="deploy_flow_with_rrule_scheduler.py:main_flow")
     .deploy(name="deploy_flow_with_rrule_scheduler",
             description="This deployment test cron scheduler",
             work_pool_name="pliu-pool",
             parameters={"book_str": test_str},
             # Run every weekday at 9 AM
             rrule="FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;BYHOUR=9;BYMINUTE=0"
             ))
```

## 2. Create a deployment by using yaml config file

All the above deployment specifications can be encapsulated inside a `prefect.yaml`.

The below `prefect.yaml` is an example

```yaml
# prefect.yaml
name: deployment_in_prefect
prefect-version: 3.4.24

# it can have multiple deployments
deployments:
  # Deployment 1: ETL flow
  - name: deploy_flow_with_yaml
    version: "1.0.0"
    tags: [test, local]
    description: "This deployment deploy a workflow via prefect.yaml"
    entrypoint: flows/deploy_flow_with_params.py:main_flow
    work_pool:
      name: pliu-pool
      work_queue_name: default
      type: process
      job_variables:
        env:
          PREFECT_LOGGING_LEVEL: INFO
    schedule:
      interval: 900        # seconds (900s = 15 minutes)
      anchor_date: "2025-11-03T00:00:00Z"  # optional: start reference time (UTC)
      timezone: "Europe/Paris"             # optional: local timezone
      active: true                         # enable/disable scheduling

```

To register the deployment, you need to run the below command

```powershell
prefect deploy

# this will prompt the below output
 Would you like to use an existing deployment configuration? [Use arrows to move; enter to select; n to select none]
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃    ┃ Name                  ┃ Entrypoint                                 ┃ Description                                        ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ >  │ deploy_flow_with_yaml │ flows/deploy_flow_with_params.py:main_flow │ This deployment deploy a workflow via prefect.yaml │
│    │                       │                                            │ No, configure a new deployment                     │
└────┴───────────────────────┴────────────────────────────────────────────┴────────────────────────────────────────────────────┘
? Your Prefect workers will need access to this flow's code in order to run it. Would you like your workers to pull your flow code from a remote storage location when running this flow? [y/n] (y): n
```

> After you finished the prompt, you should see a new deployment created in the prefect server
>
The below figure shows an example

![deployment_with_yaml.png](../assets/deployment_with_yaml.png)

